In [1]:
import subprocess
from pathlib import Path
from datetime import datetime, timedelta
import re
from collections import defaultdict


In [2]:
def extract_lines(logfile: Path, keyword: str) -> list:
    try:
        result = subprocess.check_output(f"tf -g < {logfile}", shell=True, text=True, stderr=subprocess.DEVNULL)
        return [line for line in result.splitlines() if keyword in line and "FUT_TAIFEX_TXF:" in line]
    except:
        return []


In [3]:
def extract_fire_fill_events(date: str):
    base_path = Path("/nfs/datafiles.optiontraderlogs") / date.replace("-", "/")
    logs = {
        "5F": base_path / "capital_neutrino_txf_5f" / f"output.neutrino_txf_5f.{date.replace('-', '')}.log",
        "6F": base_path / "capital_neutrino_txf" / f"output.neutrino_txf.{date.replace('-', '')}.log"
    }

    fire_by_month = defaultdict(list)
    fill_by_floor = defaultdict(list)

    for floor, logfile in logs.items():
        fire_lines = extract_lines(logfile, "[FIRE]")
        fill_lines = extract_lines(logfile, "[FILL]")

        for line in fire_lines:
            if "[FIRE DONE]" in line:
                continue
            m = re.search(r"FUT_TAIFEX_TXF:(\d+)", line)
            if not m:
                continue
            try:
                ts = datetime.strptime(line[:15], "%H:%M:%S.%f")
                month = m.group(1)
                fire_by_month[month].append((floor, ts, line))
            except:
                continue

        for line in fill_lines:
            try:
                ts = datetime.strptime(line[:15], "%H:%M:%S.%f")
                fill_by_floor[floor].append((ts, line))
            except:
                continue

    return fire_by_month, fill_by_floor, logs


In [4]:
def group_matched_fires(fire_by_month: dict, threshold_ms=1) -> dict:
    matched = {}
    for month, events in fire_by_month.items():
        events.sort(key=lambda x: x[1])
        i = 0
        pairs = []
        while i < len(events) - 1:
            f1, t1, l1 = events[i]
            f2, t2, l2 = events[i+1]
            diff = abs((t2 - t1).total_seconds() * 1000)
            if f1 != f2 and diff <= threshold_ms:
                pairs.append([(f1, t1, l1), (f2, t2, l2)])
                i += 2
            else:
                i += 1
        if pairs:
            matched[month] = pairs
    return matched


In [5]:
def match_fills(fire_events: list, fill_events: list, max_gap_ms=5) -> dict:
    result = {}
    for floor, fire_time, fire_line in fire_events:
        possible_fills = [line for (ts, line) in fill_events if 0 <= (ts - fire_time).total_seconds() * 1000 <= max_gap_ms]
        if possible_fills:
            first_fill_time = datetime.strptime(possible_fills[0][:15], "%H:%M:%S.%f")
            result[(floor, fire_time, fire_line)] = (first_fill_time, possible_fills[0])
    return result


In [6]:
def print_matched_summary(matched, fill_map):
    print(f"\n=== Matched FIRE Comparison ===")
    for month, pairs in matched.items():
        print(f"\n=== Contract {month} ===")
        for group in pairs:
            group_sorted = sorted(group, key=lambda x: x[1])
            faster, slower = group_sorted[0][0], group_sorted[1][0]
            diff_ms = abs((group_sorted[1][1] - group_sorted[0][1]).total_seconds() * 1000)
            print("---")
            print(f"{faster} faster than {slower} by {diff_ms:.3f} ms")
            for f, t, line in group_sorted:
                print(f"{f} {t.time()} {line}")

            # FIRE → FILL latency
            f1 = tuple(group_sorted[0])
            f2 = tuple(group_sorted[1])
            if f1 in fill_map and f2 in fill_map:
                lat1 = (fill_map[f1][0] - f1[1]).total_seconds() * 1000
                lat2 = (fill_map[f2][0] - f2[1]).total_seconds() * 1000
                if lat1 < lat2:
                    print(f"{f1[0]} faster from FIRE to FILL by {abs(lat2 - lat1):.3f} ms")
                else:
                    print(f"{f2[0]} faster from FIRE to FILL by {abs(lat2 - lat1):.3f} ms")
                print(f"{f1[0]} FIRE: {f1[1].time()}  FILL: {fill_map[f1][0].time()}")
                print(f"{f2[0]} FIRE: {f2[1].time()}  FILL: {fill_map[f2][0].time()}")
            else:
                print("One or both FIREs have no matching FILL")


In [7]:
# ✅ 替換這裡的日期
date_str = "2025-06-27"

# 抽出資料
fire_by_month, fill_by_floor, logs = extract_fire_fill_events(date_str)

# 比對兩樓層 FIRE 時間接近的 pair
matched_fires = group_matched_fires(fire_by_month)

# 把所有 FIRE 與 FILL 各樓層分別比對
fire_all = [f for lst in fire_by_month.values() for f in lst]
fill_all = fill_by_floor["5F"] + fill_by_floor["6F"]
fill_map = match_fills(fire_all, fill_all)

# 最後比對與輸出
print_matched_summary(matched_fires, fill_map)



=== Matched FIRE Comparison ===

=== Contract 202512 ===
---
6F faster than 5F by 0.018 ms
6F 10:35:50.503932 10:35:50.503932 FUT_TAIFEX_TXF:202512 [FIRE] dir:sell px:21639 sz:1 m:23@21639x21645@4 bid_fire_offset:0 ask_fire_offset:0 h:8@21629x21635@4 edge:-16x4 msglag:0.000028
5F 10:35:50.503950 10:35:50.503950 FUT_TAIFEX_TXF:202512 [FIRE] dir:sell px:21639 sz:1 m:23@21639x21645@4 bid_fire_offset:0 ask_fire_offset:0 h:8@21629x21635@4 edge:-16x4 msglag:0.000042
5F faster from FIRE to FILL by 0.018 ms
6F FIRE: 10:35:50.503932  FILL: 10:35:50.505807
5F FIRE: 10:35:50.503950  FILL: 10:35:50.505807
---
6F faster than 5F by 0.015 ms
6F 10:35:50.530916 10:35:50.530916 FUT_TAIFEX_TXF:202512 [FIRE] dir:sell px:21639 sz:1 m:21@21639x21643@5 bid_fire_offset:0 ask_fire_offset:0 h:8@21628x21635@4 edge:-15x4 msglag:0.000150
5F 10:35:50.530931 10:35:50.530931 FUT_TAIFEX_TXF:202512 [FIRE] dir:sell px:21639 sz:1 m:21@21639x21643@5 bid_fire_offset:0 ask_fire_offset:0 h:8@21628x21635@4 edge:-15x4 msglag

In [ ]:
import subprocess
from pathlib import Path
from datetime import datetime, timedelta
import re
from collections import defaultdict

def extract_lines(logfile: Path, keyword: str) -> list:
    try:
        result = subprocess.check_output(f"tf -g < {logfile}", shell=True, text=True, stderr=subprocess.DEVNULL)
        return [line for line in result.splitlines() if keyword in line and "FUT_TAIFEX_TXF:" in line]
    except:
        return []

def extract_fire_fill_events(date: str):
    base_path = Path("/nfs/datafiles.optiontraderlogs") / date.replace("-", "/")
    logs = {
        "5F": base_path / "capital_neutrino_txf_5f" / f"output.neutrino_txf_5f.{date.replace('-', '')}.log",
        "6F": base_path / "capital_neutrino_txf" / f"output.neutrino_txf.{date.replace('-', '')}.log"
    }

    fire_by_month = defaultdict(list)
    fill_by_floor = defaultdict(list)

    for floor, logfile in logs.items():
        fire_lines = extract_lines(logfile, "[FIRE]")
        fill_lines = extract_lines(logfile, "[FILL]")

        for line in fire_lines:
            if "[FIRE DONE]" in line:
                continue
            m = re.search(r"FUT_TAIFEX_TXF:(\d+)", line)
            if not m:
                continue
            try:
                ts = datetime.strptime(line[:15], "%H:%M:%S.%f")
                month = m.group(1)
                fire_by_month[month].append((floor, ts, line))
            except:
                continue

        for line in fill_lines:
            try:
                ts = datetime.strptime(line[:15], "%H:%M:%S.%f")
                fill_by_floor[floor].append((ts, line))
            except:
                continue

    return fire_by_month, fill_by_floor

def group_matched_fires(fire_by_month: dict, threshold_ms=1) -> dict:
    matched = {}
    for month, events in fire_by_month.items():
        events.sort(key=lambda x: x[1])
        i = 0
        pairs = []
        while i < len(events) - 1:
            f1, t1, l1 = events[i]
            f2, t2, l2 = events[i+1]
            diff = abs((t2 - t1).total_seconds() * 1000)
            if f1 != f2 and diff <= threshold_ms:
                pairs.append([(f1, t1, l1), (f2, t2, l2)])
                i += 2
            else:
                i += 1
        if pairs:
            matched[month] = pairs
    return matched

def match_fills(fire_events: list, fill_events: list, max_gap_ms=20) -> dict:
    result = {}
    for floor, fire_time, fire_line in fire_events:
        possible_fills = [line for (ts, line) in fill_events if 0 <= (ts - fire_time).total_seconds() * 1000 <= max_gap_ms]
        if possible_fills:
            first_fill_time = datetime.strptime(possible_fills[0][:15], "%H:%M:%S.%f")
            result[(floor, fire_time, fire_line)] = (first_fill_time, possible_fills[0])
    return result

def compare_fire_fill(start_date: str, end_date: str):
    d0 = datetime.strptime(start_date, "%Y-%m-%d").date()
    d1 = datetime.strptime(end_date, "%Y-%m-%d").date()

    summary = {
        "total_matched": 0,
        "5F_fire_faster": 0,
        "6F_fire_faster": 0,
        "5F_fill_latency_total": 0.0,
        "6F_fill_latency_total": 0.0,
        "5F_fill_faster": 0,
        "6F_fill_faster": 0,
    }

    while d0 <= d1:
        try:
            fire_by_month, fill_by_floor = extract_fire_fill_events(str(d0))
            matched = group_matched_fires(fire_by_month)
            fire_all = [f for lst in fire_by_month.values() for f in lst]
            fill_all = fill_by_floor["5F"] + fill_by_floor["6F"]
            fill_map = match_fills(fire_all, fill_all)

            for month, pairs in matched.items():
                for group in pairs:
                    summary["total_matched"] += 1
                    group_sorted = sorted(group, key=lambda x: x[1])
                    f1, f2 = tuple(group_sorted[0]), tuple(group_sorted[1])

                    if f1[0] == "5F":
                        summary["5F_fire_faster"] += 1
                    else:
                        summary["6F_fire_faster"] += 1

                    if f1 in fill_map and f2 in fill_map:
                        lat1 = (fill_map[f1][0] - f1[1]).total_seconds() * 1000
                        lat2 = (fill_map[f2][0] - f2[1]).total_seconds() * 1000

                        summary["5F_fill_latency_total"] += lat1 if f1[0] == "5F" else lat2
                        summary["6F_fill_latency_total"] += lat2 if f2[0] == "6F" else lat1

                        if lat1 < lat2:
                            if f1[0] == "5F":
                                summary["5F_fill_faster"] += 1
                            else:
                                summary["6F_fill_faster"] += 1
                        else:
                            if f2[0] == "5F":
                                summary["5F_fill_faster"] += 1
                            else:
                                summary["6F_fill_faster"] += 1

        except Exception as e:
            print(f"Error on {d0}: {e}")
        d0 += timedelta(days=1)

    return summary


In [17]:
compare_fire_fill("2025-06-1", "2025-06-27")

{'total_matched': 30,
 '5F_fire_faster': 3,
 '6F_fire_faster': 27,
 '5F_fill_latency_total': 13.120999999999997,
 '6F_fill_latency_total': 13.574000000000002,
 '5F_fill_faster': 14,
 '6F_fill_faster': 1}

In [14]:
def compare_fire_fill_detailed_with_split_summary(start_date: str, end_date: str):
    d0 = datetime.strptime(start_date, "%Y-%m-%d").date()
    d1 = datetime.strptime(end_date, "%Y-%m-%d").date()

    fire_count_5f = 0
    fire_count_6f = 0
    fire_total_5f_ms = 0.0
    fire_total_6f_ms = 0.0

    fill_count_5f = 0
    fill_count_6f = 0
    fill_total_5f_ms = 0.0
    fill_total_6f_ms = 0.0

    output_lines = []

    while d0 <= d1:
        date_str = str(d0)
        try:
            fire_by_month, fill_by_floor = extract_fire_fill_events(date_str)
            matched = group_matched_fires(fire_by_month)
            fire_all = [f for lst in fire_by_month.values() for f in lst]
            fill_all = fill_by_floor["5F"] + fill_by_floor["6F"]
            fill_map = match_fills(fire_all, fill_all)

            for month, pairs in matched.items():
                output_lines.append(f"\n=== Matched FIRE signal for contract {month} ===")
                for group in pairs:
                    group_sorted = sorted(group, key=lambda x: x[1])
                    f1, t1, l1 = group_sorted[0]
                    f2, t2, l2 = group_sorted[1]
                    diff_ms = abs((t2 - t1).total_seconds() * 1000)
                    fire_faster = f1 if t1 < t2 else f2
                    fire_slower = f2 if t1 < t2 else f1

                    output_lines.append("---")
                    output_lines.append(f"{fire_faster} faster than {fire_slower} by {diff_ms:.3f} ms")

                    # FIRE detail
                    for floor, ts, line in group_sorted:
                        output_lines.append(f"{floor} {ts.time()} {line}")
                        fill_info = fill_map.get((floor, ts, line))
                        if fill_info:
                            fill_ts = fill_info[0]
                            latency = (fill_ts - ts).total_seconds() * 1000
                            output_lines.append(f"   → FILL at {fill_ts.time()} (delay: {latency:.3f} ms)")
                        else:
                            output_lines.append("   → NO FILL")

                    # FIRE timing summary
                    if fire_faster == "5F":
                        fire_count_5f += 1
                        fire_total_5f_ms += diff_ms
                    else:
                        fire_count_6f += 1
                        fire_total_6f_ms += diff_ms

                    # FILL timing summary
                    f1_key = (f1, t1, l1)
                    f2_key = (f2, t2, l2)
                    if f1_key in fill_map and f2_key in fill_map:
                        lat1 = (fill_map[f1_key][0] - t1).total_seconds() * 1000
                        lat2 = (fill_map[f2_key][0] - t2).total_seconds() * 1000
                        if lat1 < lat2:
                            if f1 == "5F":
                                fill_count_5f += 1
                                fill_total_5f_ms += lat2 - lat1
                            else:
                                fill_count_6f += 1
                                fill_total_6f_ms += lat1 - lat2
                        else:
                            if f2 == "5F":
                                fill_count_5f += 1
                                fill_total_5f_ms += lat1 - lat2
                            else:
                                fill_count_6f += 1
                                fill_total_6f_ms += lat2 - lat1
        except Exception as e:
            output_lines.append(f"\n=== Error on {date_str}: {e}")
        d0 += timedelta(days=1)

    output_lines.append("\n===== FINAL SUMMARY =====")
    output_lines.append("FIRE timing comparison:")
    output_lines.append(f"6F faster count: {fire_count_6f}")
    output_lines.append(f"6F average faster: {fire_total_6f_ms / fire_count_6f:.3f} ms" if fire_count_6f else "6F average faster: N/A")
    output_lines.append(f"5F faster count: {fire_count_5f}")
    output_lines.append(f"5F average faster: {fire_total_5f_ms / fire_count_5f:.3f} ms" if fire_count_5f else "5F average faster: N/A")

    output_lines.append("\nFILL timing comparison:")
    output_lines.append(f"6F faster count: {fill_count_6f}")
    output_lines.append(f"6F average faster: {fill_total_6f_ms / fill_count_6f:.3f} ms" if fill_count_6f else "6F average faster: N/A")
    output_lines.append(f"5F faster count: {fill_count_5f}")
    output_lines.append(f"5F average faster: {fill_total_5f_ms / fill_count_5f:.3f} ms" if fill_count_5f else "5F average faster: N/A")

    return "\n".join(output_lines)



In [16]:
print(compare_fire_fill_detailed_with_split_summary("2025-06-20", "2025-06-30"))



=== Matched FIRE signal for contract 202507 ===
---
6F faster than 5F by 0.026 ms
6F 08:45:00.054461 08:45:00.054461 FUT_TAIFEX_TXF:202507 [FIRE] dir:sell px:21635 sz:1 m:3@21635x21636@1 bid_fire_offset:0 ask_fire_offset:0 h:9@21628x21630@21 edge:-8x5 msglag:0.000099
   → NO FILL
5F 08:45:00.054487 08:45:00.054487 FUT_TAIFEX_TXF:202507 [FIRE] dir:sell px:21635 sz:1 m:3@21635x21636@1 bid_fire_offset:0 ask_fire_offset:0 h:9@21628x21630@21 edge:-8x5 msglag:0.000112
   → NO FILL
---
6F faster than 5F by 0.019 ms
6F 08:45:00.070083 08:45:00.070083 FUT_TAIFEX_TXF:202507 [FIRE] dir:sell px:21635 sz:1 m:3@21635x21636@1 bid_fire_offset:0 ask_fire_offset:0 h:7@21628x21630@21 edge:-8x5 msglag:0.000016
   → FILL at 08:45:00.071239 (delay: 1.156 ms)
5F 08:45:00.070102 08:45:00.070102 FUT_TAIFEX_TXF:202507 [FIRE] dir:sell px:21635 sz:1 m:3@21635x21636@1 bid_fire_offset:0 ask_fire_offset:0 h:7@21628x21630@21 edge:-8x5 msglag:0.000028
   → FILL at 08:45:00.071239 (delay: 1.137 ms)

=== Matched FIRE s

In [23]:
from pathlib import Path
from datetime import datetime
import subprocess, re
from collections import defaultdict

def extract_fire_and_fill(date: str):
    base = Path("/nfs/datafiles.optiontraderlogs") / date.replace("-", "/")
    logs = {
        "5F": base / "capital_neutrino_txf_5f" / f"output.neutrino_txf_5f.{date.replace('-', '')}.log",
        "6F": base / "capital_neutrino_txf" / f"output.neutrino_txf.{date.replace('-', '')}.log",
    }
    fire_by_month = defaultdict(list)
    fills = defaultdict(list)

    for floor, log in logs.items():
        try:
            lines = subprocess.check_output(f"tf -g < {log}", shell=True, text=True, stderr=subprocess.DEVNULL).splitlines()
        except:
            continue
        for line in lines:
            if "FUT_TAIFEX_TXF:" not in line: continue
            try: ts = datetime.strptime(line[:15], "%H:%M:%S.%f")
            except: continue
            if "[FIRE]" in line and "[FIRE DONE]" not in line:
                m = re.search(r"FUT_TAIFEX_TXF:(\d+)", line)
                if m: fire_by_month[m.group(1)].append((floor, ts, line))
            elif "[FILL]" in line:
                fills[floor].append((ts, line))

    for lst in fire_by_month.values():
        lst.sort(key=lambda x: x[1])
    return fire_by_month, fills

def match_pairs(fire_events, threshold_ms=1):
    pairs = []
    i = 0
    while i < len(fire_events) - 1:
        a, b = fire_events[i], fire_events[i+1]
        if a[0] != b[0]:
            diff = abs((b[1] - a[1]).total_seconds() * 1000)
            if diff <= threshold_ms:
                pairs.append(sorted([a, b], key=lambda x: x[1]))
                i += 2
                continue
        i += 1
    return pairs

def find_fill(ts, fill_list, max_delay=20):
    for fill_ts, line in fill_list:
        delay = (fill_ts - ts).total_seconds() * 1000
        if 0 <= delay <= max_delay:
            return fill_ts, delay
    return None, None

def print_day_summary(date: str):
    fire_by_month, fills = extract_fire_and_fill(date)
    fire_stat = {"5F": 0, "6F": 0, "dt": {"5F": 0.0, "6F": 0.0}}
    fill_stat = {"5F": 0, "6F": 0, "dt": {"5F": 0.0, "6F": 0.0}}

    print(f"\n=== Matched FIRE signal for {date} ===")
    for month, events in fire_by_month.items():
        pairs = match_pairs(events)
        if not pairs: continue
        print(f"\n=== Matched FIRE signal for contract {month} ===")
        for (f1, t1, l1), (f2, t2, l2) in pairs:
            diff = abs((t2 - t1).total_seconds() * 1000)
            faster = f1 if t1 < t2 else f2
            slower = f2 if t1 < t2 else f1
            print("---")
            print(f"{faster} faster than {slower} by {diff:.3f} ms")

            # Print both FIRE + fill
            for f, t, line in [(f1, t1, l1), (f2, t2, l2)]:
                print(f"{f} {t.time()} {t.time()} {line}")
                fill_ts, delay = find_fill(t, fills[f])
                if fill_ts:
                    print(f"   → FILL at {fill_ts.time()} (delay: {delay:.3f} ms)")
                else:
                    print("   → NO FILL")

            # Update FIRE stats
            fire_stat[faster] += 1
            fire_stat["dt"][faster] += diff

            # Compare FILL only if both present
            fill_1, delay_1 = find_fill(t1, fills[f1])
            fill_2, delay_2 = find_fill(t2, fills[f2])
            if fill_1 and fill_2:
                if delay_1 < delay_2:
                    fill_stat[f1] += 1
                    fill_stat["dt"][f1] += delay_2 - delay_1
                elif delay_2 < delay_1:
                    fill_stat[f2] += 1
                    fill_stat["dt"][f2] += delay_1 - delay_2
                # 如果一樣快，不記任何人

    # Summary
    print("\n===== FINAL SUMMARY =====")
    print("FIRE timing comparison:")
    for f in ["6F", "5F"]:
        count = fire_stat[f]
        total = fire_stat["dt"][f]
        print(f"{f} faster count: {count}")
        print(f"{f} average faster: {total / count:.3f} ms" if count else f"{f} average faster: N/A")

    print("\nFILL timing comparison:")
    for f in ["6F", "5F"]:
        count = fill_stat[f]
        total = fill_stat["dt"][f]
        print(f"{f} faster count: {count}")
        print(f"{f} average faster: {total / count:.3f} ms" if count else f"{f} average faster: N/A")


In [24]:
print_day_summary("2025-06-20")



=== Matched FIRE signal for 2025-06-20 ===

=== Matched FIRE signal for contract 202507 ===
---
6F faster than 5F by 0.026 ms
6F 08:45:00.054461 08:45:00.054461 08:45:00.054461 FUT_TAIFEX_TXF:202507 [FIRE] dir:sell px:21635 sz:1 m:3@21635x21636@1 bid_fire_offset:0 ask_fire_offset:0 h:9@21628x21630@21 edge:-8x5 msglag:0.000099
   → FILL at 08:45:00.063471 (delay: 9.010 ms)
5F 08:45:00.054487 08:45:00.054487 08:45:00.054487 FUT_TAIFEX_TXF:202507 [FIRE] dir:sell px:21635 sz:1 m:3@21635x21636@1 bid_fire_offset:0 ask_fire_offset:0 h:9@21628x21630@21 edge:-8x5 msglag:0.000112
   → FILL at 08:45:00.064117 (delay: 9.630 ms)
---
6F faster than 5F by 0.019 ms
6F 08:45:00.070083 08:45:00.070083 08:45:00.070083 FUT_TAIFEX_TXF:202507 [FIRE] dir:sell px:21635 sz:1 m:3@21635x21636@1 bid_fire_offset:0 ask_fire_offset:0 h:7@21628x21630@21 edge:-8x5 msglag:0.000016
   → FILL at 08:45:00.071365 (delay: 1.282 ms)
5F 08:45:00.070102 08:45:00.070102 08:45:00.070102 FUT_TAIFEX_TXF:202507 [FIRE] dir:sell px:

In [25]:
def process_range_summary(start_date: str, end_date: str):
    from datetime import datetime, timedelta

    d0 = datetime.strptime(start_date, "%Y-%m-%d").date()
    d1 = datetime.strptime(end_date, "%Y-%m-%d").date()

    fire_stat = {"5F": 0, "6F": 0, "dt": {"5F": 0.0, "6F": 0.0}}
    fill_stat = {"5F": 0, "6F": 0, "dt": {"5F": 0.0, "6F": 0.0}}

    while d0 <= d1:
        date_str = str(d0)
        try:
            fire_by_month, fills = extract_fire_and_fill(date_str)
            for month, events in fire_by_month.items():
                pairs = match_pairs(events)
                for (f1, t1, l1), (f2, t2, l2) in pairs:
                    diff = abs((t2 - t1).total_seconds() * 1000)
                    faster = f1 if t1 < t2 else f2
                    fire_stat[faster] += 1
                    fire_stat["dt"][faster] += diff

                    fill_1, delay_1 = find_fill(t1, fills[f1])
                    fill_2, delay_2 = find_fill(t2, fills[f2])
                    if fill_1 and fill_2:
                        if delay_1 < delay_2:
                            fill_stat[f1] += 1
                            fill_stat["dt"][f1] += delay_2 - delay_1
                        elif delay_2 < delay_1:
                            fill_stat[f2] += 1
                            fill_stat["dt"][f2] += delay_1 - delay_2
        except Exception as e:
            print(f"Error on {date_str}: {e}")
        d0 += timedelta(days=1)

    print("\n===== FINAL SUMMARY (ALL DAYS) =====")
    print("FIRE timing comparison:")
    for f in ["6F", "5F"]:
        count = fire_stat[f]
        total = fire_stat["dt"][f]
        print(f"{f} faster count: {count}")
        print(f"{f} average faster: {total / count:.3f} ms" if count else f"{f} average faster: N/A")

    print("\nFILL timing comparison:")
    for f in ["6F", "5F"]:
        count = fill_stat[f]
        total = fill_stat["dt"][f]
        print(f"{f} faster count: {count}")
        print(f"{f} average faster: {total / count:.3f} ms" if count else f"{f} average faster: N/A")


In [26]:
process_range_summary("2025-06-1", "2025-06-30")



===== FINAL SUMMARY (ALL DAYS) =====
FIRE timing comparison:
6F faster count: 27
6F average faster: 0.026 ms
5F faster count: 3
5F average faster: 0.350 ms

FILL timing comparison:
6F faster count: 13
6F average faster: 0.882 ms
5F faster count: 9
5F average faster: 0.771 ms
